In [0]:
pip install azure-eventhub faker

In [0]:
import json
import random
from datetime import datetime, timedelta
from faker import Faker
from azure.eventhub import EventHubProducerClient, EventData

# ----------------------------
# Azure Event Hub Connection
# ----------------------------
CONNECTION_STR = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-usage-policy")
EVENTHUB_NAME = dbutils.secrets.get(scope = "telco-secret-keys", key = "telco-usage-hub")

# ----------------------------
# Initialize Faker
# ----------------------------
fake = Faker()

# Customer IDs 1000–1100
CUSTOMER_IDS = list(range(1000, 1101))

# ----------------------------
# Generate Usage Record
# ----------------------------
def generate_usage(customer_id):
    return {
        "UsageID": f"U{random.randint(10000, 99999)}",
        "CustomerID": customer_id,
        "UsageDate": (datetime.now() - timedelta(days=random.randint(0, 30))).strftime("%Y-%m-%d"),
        "DataUsedMB": random.randint(100, 5000),       # 100MB – 5GB
        "CallMinutes": random.randint(0, 500),         # 0 – 500 mins
        "SMSCount": random.randint(0, 200),            # 0 – 200 SMS
        "RoamingMinutes": random.randint(0, 100)       # 0 – 100 mins
    }

# ----------------------------
# Send to Event Hub
# ----------------------------
def send_usage_events(num_events=50):
    producer = EventHubProducerClient.from_connection_string(
        conn_str=CONNECTION_STR,
        eventhub_name=EVENTHUB_NAME
    )
    event_batch = producer.create_batch()

    for _ in range(num_events):
        customer_id = random.choice(CUSTOMER_IDS)
        usage_record = generate_usage(customer_id)
        event_data = EventData(json.dumps(usage_record))
        event_batch.add(event_data)

    producer.send_batch(event_batch)
    print(f"✅ Sent {num_events} usage events to Event Hub")

# ----------------------------
# Run
# ----------------------------
if __name__ == "__main__":
    send_usage_events(100)  # Send 100 usage events
